# Necessary packages

In [ ]:
!pip install soundfile
!pip install espnet_model_zoo
!pip install python-Levenshtein
!pip install espnet

# Test set (Librispeech test-clean) for default model

In [ ]:
!wget http://www.openslr.org/resources/12/test-clean.tar.gz
!tar -zxvf test-clean.tar.gz

In [ ]:
!find . \( -iname '*.flac' -o -iname '*.trans.txt' \) -type f | sort | sed 's/^..//' > files-all.txt
#finds all flac and transcript files -> sorts -> removes first 2 chars (./) from all lines -> stores all this in files-all.txt

# Imports

In [ ]:
import soundfile
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text
import numpy as np
from torch.utils.data import Dataset, DataLoader
import re
from Levenshtein import distance
from tqdm.notebook import tqdm

# Download model for testing

In [ ]:
d = ModelDownloader()

libriModel = "Shinji Watanabe/librispeech_asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best"
speech2text = Speech2Text(**d.download_and_unpack(libriModel))

# Dataset/Dataloader

In [ ]:
nameRegex = re.compile('[0-9]+-[0-9]+-[0-9]+ ') #in transcript files matches number preceding text


class MyDataset(Dataset):
    def __init__(self, paths):
      super().__init__()
      raw = np.loadtxt(paths,delimiter='\n',dtype=str)
      delInd = []
      self.labels = []
      for i,s in enumerate(raw):
        if s.endswith(".txt"):
          delInd.append(i)
          rawL = np.loadtxt(s,delimiter='\n',dtype=str)
          for i in range(len(rawL)):
            rawL[i] = re.sub(nameRegex, '', rawL[i])
          self.labels.append(rawL)


      self.data = np.delete(raw,delInd)
      self.labels = np.concatenate(self.labels)

      assert len(self.data) == len(self.labels)
    def __len__(self):

      return self.data.shape[0]

    def __getitem__(self, ind):
      return soundfile.read(self.data[ind])[0], self.labels[ind], self.data[ind]

testSet = MyDataset('files-all.txt')

In [ ]:
dist = 0
numSamp = 0
for (sound, transcript, filename) in testSet:
  nb = speech2text(sound)
  text, *_ = nbests[0]
  dist += distance(text, transcript)
  numSamp += len(text)

In [ ]:
print('Average Levenshtein Distance for Test Set: %.1f' % (dist/numSamp))

Average Levenshtein Distance for Test Set: 1.0
